# B''H

## Examining Racial Discrimination in the US Job Market

In [3]:
import os
import sys

import math

import numpy as np
import pandas as pd

from scipy import stats

import matplotlib.pyplot as plt

import seaborn as sns

---
## Set the plot output sizes

In [4]:
# Get current size
fig_size = plt.rcParams["figure.figsize"]
 
# Prints: [8.0, 6.0]
print ("Prior size:", fig_size)
 
# Set figure width to 12 and height to 9
fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

print ("Current size:", fig_size)

Prior size: [6.0, 4.0]
Current size: [12, 9]


---
## Get project info

In [5]:
NOTEBOOKS_DIR = os.path.join(os.pardir)

print(os.path.abspath(NOTEBOOKS_DIR))

/home/laz/repos/springboard-mini-projects/notebooks


In [6]:
PROJ_ROOT = os.path.join(NOTEBOOKS_DIR,os.pardir)

print(os.path.abspath(PROJ_ROOT))

/home/laz/repos/springboard-mini-projects


In [7]:
# add the 'src' directory as one where we can import modules
SRC_DIR = os.path.join(PROJ_ROOT, 'src')
sys.path.append(SRC_DIR)

print(os.path.abspath(SRC_DIR))

/home/laz/repos/springboard-mini-projects/src


In [8]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

# import my method from the source code
%aimport helper_functions
import helper_functions as hf

---

---

### Background
- Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
- In the dataset provided, each row represents a resume. 
- The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. 
- The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.
- Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

## Exercises
- Perform statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

---
## Import the data:

In [9]:
v_file = os.path.join(hf.DATA_DIR, 'us_job_market_discrimination.dta')

job_data = pd.io.stata.read_stata(v_file)

job_data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [10]:
job_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [12]:
job_data['race'].unique()

array(['w', 'b'], dtype=object)

In [13]:
job_data['call'].unique()

array([0., 1.])

In [19]:
race_call = job_data.loc[:, ['race', 'call']]

race_call.head()

,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,b,0.0
4,w,0.0


In [20]:
race_call.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 2 columns):
race    4870 non-null object
call    4870 non-null float32
dtypes: float32(1), object(1)
memory usage: 95.1+ KB


In [32]:
f = {'call':['count','sum']}

race_call.groupby('race').agg(f)

call       
     count    sum
race             
b     2435  157.0
w     2435  235.0

In [41]:
black_calls = race_call[race_call['race'] == 'b']['call'].values
white_calls = race_call[race_call['race'] == 'w']['call'].values

print(len(black_calls), len(white_calls), np.sum(black_calls), np.sum(white_calls))

2435 2435 157.0 235.0


In [43]:
white_callback_rate = np.sum(white_calls) / len(white_calls)
black_callback_rate = np.sum(black_calls) / len(black_calls)

white_callback_rate, black_callback_rate

(0.09650924024640657, 0.06447638603696099)

---
## **Questions 1 and 2**

**Question 1:** What test is appropriate for this problem? Does CLT apply?

**Answer:**
- This would be a good scenario for using an A/B Test
- The null hypotyhesis is that call-back rate is not affected by the race
- The Central Limit Theorem (CLT) states that regardless of the population, the distribution of sample averages tends to be normal. This holds for sample sizes greater than or equal to 30.

--- 

**Question 2:** What are the null and alternate hypotheses?

**Answer:**
- The null hypotyhesis is that call-back rate is not affected by the race

---
### Perform A/B Test

In [40]:
diff_frac_observed = hf.diff_frac(white_calls, black_calls)

diff_frac_observed

0.032032854209445585

In [52]:
# Acquire permutation samples: perm_replicates
perm_replicates = hf.draw_permutation_replicates(
    white_calls, 
    black_calls, 
    hf.diff_frac, 
    10000
)


In [53]:
# Compute and print p-value: p
p = np.sum(perm_replicates >= diff_frac_observed) / len(perm_replicates)
print('p-value =', p)

p-value = 0.0


### Result:
- The p-value is 0, so the null hypothesis is rejected. 
- There is sufficient evidence to warrant rejection of the claim that there is no difference in the proportion of callbacks for white-sounding names and the proportion of callbacks for black-sounding names. 

---
### Get the 95% Confidence Interval

In [55]:
np.percentile(perm_replicates, [2.5, 97.5])

array([-0.01478439,  0.01560575])

### Result:
- The confidence interval does not contain the `diff_frac_observed` value of `0.032`
- Again this is sufficient evidence to warrant rejection of the claim that there is no difference in the proportion of callbacks for white-sounding names and the proportion of callbacks for black-sounding names. 

---
### Use the frequentist statistical approach

In [58]:
# Compute and save confidence interval and margin of error
difference_of_proportions, moe, confidence_lower, confidence_higher = hf.two_proportion_confidence_interval(
    hits1     = np.sum(white_calls), 
    attempts1 = len(white_calls), 
    hits2     = np.sum(black_calls),
    attempts2 = len(black_calls), 
    alpha     = 0.05
)

print('Difference of Proportions : {0:0.4f}'.format(difference_of_proportions))
print('Margin of Error           : {0:0.4f}'.format(moe))
print('95% Confidence Interval   : [ {0:0.4f}  {1:0.4f} ]'.format(confidence_lower, confidence_higher))

Difference of Proportions : 0.0320
Margin of Error           : 0.0153
95% Confidence Interval   : [ -0.0473  -0.0168 ]


---
## **Question 3**
- Does your analysis mean that race/name is the most important factor in callback success? 
- Why or why not? 
- If not, how would you amend your analysis?


**Answer**
- This analysis does not mean that race is the most important factor in callback success. 
- Other factors, such as gender, have not been tested.
- A look at the correlation of all categorical variables could also be helpful. 
- However, some of the variables are not clear, so more information is needed.